In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/competitive-data-science-predict-future-sales/items.csv
/kaggle/input/competitive-data-science-predict-future-sales/sample_submission.csv
/kaggle/input/competitive-data-science-predict-future-sales/item_categories.csv
/kaggle/input/competitive-data-science-predict-future-sales/sales_train.csv
/kaggle/input/competitive-data-science-predict-future-sales/shops.csv
/kaggle/input/competitive-data-science-predict-future-sales/test.csv


In [2]:
df=pd.read_csv("/kaggle/input/competitive-data-science-predict-future-sales/sales_train.csv")
df=df[2885849:]

test_data=pd.read_csv("../input/competitive-data-science-predict-future-sales/test.csv")

test = test_data.drop("ID", axis=1)
df.head()


,date,date_block_num,shop_id,item_id,item_price,item_cnt_day
2885849,18.10.2015,33,42,13300,539.0,1.0
2885850,04.10.2015,33,42,11925,549.0,1.0
2885851,07.10.2015,33,42,13300,539.0,1.0
2885852,19.10.2015,33,42,13294,429.0,1.0
2885853,23.10.2015,33,42,13281,749.0,1.0


In [3]:
import pandas as pd
import numpy as np
import sklearn
from sklearn import preprocessing 
import regex as re



In [4]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn import linear_model
from sklearn import neighbors
from sklearn.preprocessing import OneHotEncoder
from xgboost import XGBClassifier

In [5]:
df=df.drop(columns=['date_block_num'], axis=1)
df['item_id'].fillna((df['item_id'].mean()), inplace=True)
df['item_id'].fillna((df['item_id'].mean()), inplace=True) 
df['item_price'].fillna((df['item_price'].mean()), inplace=True) 
df['item_cnt_day'].fillna((df['item_cnt_day'].mean()), inplace=True)

In [6]:
def add_datepart(df, fldname, drop=True, time=False):
    "Helper function that adds columns relevant to a date."
    fld = df[fldname]
    fld_dtype = fld.dtype
    if isinstance(fld_dtype, pd.core.dtypes.dtypes.DatetimeTZDtype):
        fld_dtype = np.datetime64

    if not np.issubdtype(fld_dtype, np.datetime64):
        df[fldname] = fld = pd.to_datetime(fld, infer_datetime_format=True)
    targ_pre = re.sub('[Dd]ate$', '', fldname)
    attr = ['Year', 'Month', 'Week', 'Day', 'Dayofweek', 'Dayofyear',
            'Is_month_end', 'Is_month_start', 'Is_quarter_end', 'Is_quarter_start', 'Is_year_end', 'Is_year_start']
    if time: attr = attr + ['Hour', 'Minute', 'Second']
    for n in attr: df[targ_pre + n] = getattr(fld.dt, n.lower())
    df[targ_pre + 'Elapsed'] = fld.astype(np.int64) // 10 ** 9
    if drop: df.drop(fldname, axis=1, inplace=True)

In [7]:
add_datepart(df,"date")
df=df.drop("Elapsed", axis=1)


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:14: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated.  Please use Series.dt.isocalendar().week instead.
  


In [8]:
df.head()

,shop_id,item_id,item_price,item_cnt_day,Year,Month,Week,Day,Dayofweek,Dayofyear,Is_month_end,Is_month_start,Is_quarter_end,Is_quarter_start,Is_year_end,Is_year_start
2885849,42,13300,539.0,1.0,2015,10,42,18,6,291,False,False,False,False,False,False
2885850,42,11925,549.0,1.0,2015,10,40,4,6,277,False,False,False,False,False,False
2885851,42,13300,539.0,1.0,2015,10,41,7,2,280,False,False,False,False,False,False
2885852,42,13294,429.0,1.0,2015,10,43,19,0,292,False,False,False,False,False,False
2885853,42,13281,749.0,1.0,2015,10,43,23,4,296,False,False,False,False,False,False


In [9]:
label_encoder = preprocessing.LabelEncoder()

In [10]:
df['Is_month_end']= label_encoder.fit_transform(df['Is_month_end']) 
df['Is_month_start']= label_encoder.fit_transform(df['Is_month_start']) 
df['Is_quarter_end']= label_encoder.fit_transform(df['Is_quarter_end']) 
df['Is_quarter_start']= label_encoder.fit_transform(df['Is_quarter_start']) 
df['Is_year_end']= label_encoder.fit_transform(df['Is_year_end']) 
df['Is_year_start']= label_encoder.fit_transform(df['Is_year_start'])

In [11]:
df.head()

,shop_id,item_id,item_price,item_cnt_day,Year,Month,Week,Day,Dayofweek,Dayofyear,Is_month_end,Is_month_start,Is_quarter_end,Is_quarter_start,Is_year_end,Is_year_start
2885849,42,13300,539.0,1.0,2015,10,42,18,6,291,0,0,0,0,0,0
2885850,42,11925,549.0,1.0,2015,10,40,4,6,277,0,0,0,0,0,0
2885851,42,13300,539.0,1.0,2015,10,41,7,2,280,0,0,0,0,0,0
2885852,42,13294,429.0,1.0,2015,10,43,19,0,292,0,0,0,0,0,0
2885853,42,13281,749.0,1.0,2015,10,43,23,4,296,0,0,0,0,0,0


In [12]:
test['item_price']=df['item_price']
test['item_price'].fillna((df['item_price'].mean()), inplace=True)
test['Year']=df['Year']
test['Year'].fillna((df['Year'].mean()), inplace=True)
test['Month']=df['Month']
test['Month'].fillna((df['Month'].mean()), inplace=True)
test['Week']=df['Week']
test['Week'].fillna((df['Week'].mean()), inplace=True)
test['Day']=df['Day']
test['Day'].fillna((df['Day'].mean()), inplace=True)
test['Dayofweek']=df['Dayofweek']
test['Dayofweek'].fillna((df['Dayofweek'].mean()), inplace=True)
test['Dayofyear']=df['Dayofyear']
test['Dayofyear'].fillna((df['Dayofyear'].mean()), inplace=True)
test['Is_month_end']=df['Is_month_end']
test['Is_month_end'].fillna((df['Is_month_end'].mean()), inplace=True)
test['Is_month_start']=df['Is_month_start']
test['Is_month_start'].fillna((df['Is_month_start'].mean()), inplace=True)
test['Is_quarter_end']=df['Is_quarter_end']
test['Is_quarter_end'].fillna((df['Is_quarter_end'].mean()), inplace=True)
test['Is_quarter_start']=df['Is_quarter_start']
test['Is_quarter_start'].fillna((df['Is_quarter_start'].mean()), inplace=True)
test['Is_year_end']=df['Is_year_end']
test['Is_year_end'].fillna((df['Is_year_end'].mean()), inplace=True)
test['Is_year_start']=df['Is_year_start']
test['Is_year_start'].fillna((df['Is_year_start'].mean()), inplace=True)
test.head()

,shop_id,item_id,item_price,Year,Month,Week,Day,Dayofweek,Dayofyear,Is_month_end,Is_month_start,Is_quarter_end,Is_quarter_start,Is_year_end,Is_year_start
0,5,5037,1221.149775,2015.0,10.0,42.04146,15.85972,3.5695,288.85972,0.04832,0.02982,0.0,0.02982,0.0,0.0
1,5,5320,1221.149775,2015.0,10.0,42.04146,15.85972,3.5695,288.85972,0.04832,0.02982,0.0,0.02982,0.0,0.0
2,5,5233,1221.149775,2015.0,10.0,42.04146,15.85972,3.5695,288.85972,0.04832,0.02982,0.0,0.02982,0.0,0.0
3,5,5232,1221.149775,2015.0,10.0,42.04146,15.85972,3.5695,288.85972,0.04832,0.02982,0.0,0.02982,0.0,0.0
4,5,5268,1221.149775,2015.0,10.0,42.04146,15.85972,3.5695,288.85972,0.04832,0.02982,0.0,0.02982,0.0,0.0


In [13]:
item_cnt=df['item_cnt_day']
df=df.drop(labels='item_cnt_day',axis=1)
df.insert(15,'item_cnt_day',item_cnt);

In [14]:
(80*len(df))/100

40000.0

In [15]:
train_df=df.loc[2885849:2925849]
valid_df=df.loc[2925849:]
len(train_df),len(valid_df)

(40001, 10000)

In [16]:
y_train=train_df['item_cnt_day']
x_train=train_df.drop('item_cnt_day',axis=1)
y_valid=valid_df['item_cnt_day']
x_valid=valid_df.drop('item_cnt_day',axis=1)

In [17]:
model1=RandomForestClassifier()
model2=XGBClassifier()
model3=neighbors.KNeighborsClassifier()
model4=SVC()

In [18]:
model1.fit(x_train,y_train)
model2.fit(x_train,y_train)
model3.fit(x_train,y_train)
model4.fit(x_train,y_train)

pred1=model1.predict(x_valid)
pred2=model2.predict(x_valid)
pred3=model3.predict(x_valid)
pred4=model4.predict(x_valid)

/opt/conda/lib/python3.7/site-packages/xgboost/sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[05:05:24] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


In [19]:
predictions = model4.predict(test)

submission = pd.DataFrame({
        "ID": test_data.ID,
        "item_cnt_month": predictions
    })
submission.reset_index(drop=True, inplace=True)


submission.to_csv('submission.csv', index=False)
print("Your submission was successfully saved!")

Your submission was successfully saved!


In [20]:
submission = pd.read_csv('submission.csv')
submission.head()


,ID,item_cnt_month
0,0,1.0
1,1,1.0
2,2,1.0
3,3,1.0
4,4,1.0
